# WhisperTS joining JSON to SRT

In [26]:
import re,codecs,json,unidecode
import pandas as pd

In [27]:
repJson="/Users/gilles/Github/AMBK-SportsCom/Whisper/Extrait/Transcripts/"

In [28]:
json1=repJson+"extrait-00.json"
with open(json1) as inFile:
    whisper1=json.load(inFile)

### Gestion des timestamps

In [29]:
def second2timestamp(seconds):
    sHours=int(seconds/3600)
    sMinutes=int((seconds-sHours*3600)/60)
    sSeconds=int((seconds-sHours*3600-sMinutes*60))
    sMilliseconds=int((seconds%1)*1000)
    return "%02d:%02d:%02d,%03d"%(sHours,sMinutes,sSeconds,sMilliseconds)

def timestamp2second(timestamp):
    m=re.match("(\d\d):(\d\d):(\d\d),(\d\d\d)",timestamp)
    if m:
        seconds=(int(m.group(1))*60+int(m.group(2)))*60+int(m.group(3))+float(m.group(4))/1000
        return seconds
    else:
        print("problème de timestamp")
        return

### Gestion des structures JSON de Whisper

In [30]:
def getWhisper(whisper,elements="segments"):
    lElements=[]
    if elements=="segments":
        for element in whisper[elements]:
            lElements.append((second2timestamp(element["start"]),
                              second2timestamp(element["end"]),
                              element["text"],
                              element["confidence"]))
    elif elements=="words":
        for segment in whisper["segments"]:
            for element in segment[elements]:
                lElements.append((second2timestamp(element["start"]),
                                  second2timestamp(element["end"]),
                                  element["text"],
                                  element["confidence"]))        
    return lElements

## Collage des segments

In [31]:
def removePunct(texte):
    result=texte
    punct = list('''!()-[]{};:'"\,<>./?@#$%^&*_~''')
    for p in punct:
        result=result.replace(p,"")
    return result
    
def egaleTextes(text1,text2):
    t1=unidecode.unidecode(removePunct(text1))
    t2=unidecode.unidecode(removePunct(text2))
    lMax=min(len(t1),len(t2))
    lMax=min(20,lMax)
    return t1[:lMax].lower()==t2[:lMax].lower()

In [32]:
def pasteSegment(srt,whisper2):
    for i in range(1,len(whisper2["segments"])):
        id1=None
        for n,(_,_,texte,_) in enumerate(srt):
            texte2=whisper2["segments"][i]["text"]
            if egaleTextes(texte,texte2):
                id1=n
                print(texte)
                print(texte2)
                break
        if id1:
            break
        else:
            print("problème de jointure",i, whisper2["segments"][i]["text"])
    if id1:
        jointTime=timestamp2second(srt[id1][0])
        newSrt=srt[:id1]
    
        srt2=getWhisper(whisper2)[i:]
        for (tDeb,tFin,texte,confidence) in srt2:
            nDeb=second2timestamp(timestamp2second(tDeb)+jointTime)
            nFin=second2timestamp(timestamp2second(tFin)+jointTime)
            newSrt.append((nDeb,nFin,texte,confidence))
        return newSrt
    else:
        return None

In [33]:
srtLignes=[]
for (tDeb,tFin,texte,confidence) in getWhisper(whisper1):
    nDeb=second2timestamp(timestamp2second(tDeb))
    nFin=second2timestamp(timestamp2second(tFin))
    srtLignes.append((nDeb,nFin,texte,confidence))

srtMots=[]
for (tDeb,tFin,texte,confidence) in getWhisper(whisper1,"words"):
    nDeb=second2timestamp(timestamp2second(tDeb))
    nFin=second2timestamp(timestamp2second(tFin))
    srtMots.append((nDeb,nFin,texte,confidence))


In [34]:
for i in range(1,11):
    print(i)
    json2=repJson+"extrait-%02d.json"%i
    with open(json2) as inFile:
        whisper2=json.load(inFile)
    newSrtLignes=pasteSegment(srtLignes,whisper2)
    if not newSrtLignes:
        break
    else:
        srtLignes=newSrtLignes
# print(srt)

1
 Alors la touche, ça, ça sera une des clés de cette rencontre.
 Alors la touche, ça, ça sera une des clés de cette rencontre.
2
 Les Argentins qui ont bien démarré en chipant un premier ballon à l'alignement français
 Les Argentins qui ont bien démarré en chipant un premier ballon à l'alignement français
3
 Il faut vite aller recouper le terrain pour se rassurer.
 Il faut vite aller recouper le terrain pour se rassurer parce qu'en tout cas, c'est le début de partie, il est argentin, vous le voyez.
4
 Quand on est au sol, on est un joueur mort, on ne doit plus rien faire, si ce n'est que s'écarter le plus vite possible de la zone de jeu.
 Quand on est au sol, on est un joueur mort, Thierry, on ne doit plus rien faire, si ce n'est que s'écarter le plus vite possible de la zone de jeu.
5
 et c'est vrai, vous le verrez, que Mignoni, il a un dynamisme viral, alors que Pichotte est beaucoup plus lent dans ses transitions.
 Et c'est vrai, vous le verrez, que Mignoni a un dynamisme viral alo

In [35]:
srtMots

[('00:00:04,370', '00:00:05,410', 'Spreadbury,', 0.801),
 ('00:00:05,469', '00:00:06,049', 'il', 0.954),
 ('00:00:06,049', '00:00:06,269', 'sera', 0.997),
 ('00:00:06,269', '00:00:06,929', 'encadré', 0.999),
 ('00:00:06,929', '00:00:07,030', 'par', 0.998),
 ('00:00:07,030', '00:00:07,250', 'un', 0.995),
 ('00:00:07,250', '00:00:07,730', 'Australien,', 0.927),
 ('00:00:07,809', '00:00:07,870', 'M.', 0.752),
 ('00:00:07,969', '00:00:08,390', 'Dickinson,', 0.961),
 ('00:00:08,570', '00:00:09,050', 'et', 0.954),
 ('00:00:09,050', '00:00:09,230', 'par', 0.997),
 ('00:00:09,230', '00:00:09,509', 'un', 0.999),
 ('00:00:09,509', '00:00:10,410', 'Néo-Zélandais,', 0.988),
 ('00:00:10,448', '00:00:10,849', '[*]', 0.0),
 ('00:00:10,849', '00:00:11,009', 'M.', 0.998),
 ('00:00:11,269', '00:00:11,650', 'Lawrence.', 0.941),
 ('00:00:13,380', '00:00:13,460', 'M.', 0.999),
 ('00:00:13,599', '00:00:14,460', 'Spreadbury,', 0.993),
 ('00:00:14,500', '00:00:14,900', "c'est", 1.0),
 ('00:00:14,900', '00:00:

In [37]:
srtLines=[]
for i,(start,end,mot,confidence) in enumerate(srtLignes):
    n=i+1
    srtLines.append(str(n))
    srtLines.append(start+" --> "+end)
    srtLines.append(mot)
    srtLines.append("")


In [24]:
with codecs.open(json1.replace(".json","-segments.srt"),"w",encoding="utf8") as outFile:
    for srtLine in srtLines:
        outFile.write(srtLine+"\n")